In [25]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import pymongo

In [10]:
#URL to be scraped
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)


In [11]:
soup = BeautifulSoup(response.text, 'html.parser')

In [12]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
title_results = soup.find_all('div', class_='content_title')[0]

#Print scraped article header
title_results

<div class="content_title">
<a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
NASA Ingenuity Mars Helicopter Prepares for First Flight
</a>
</div>

In [13]:
#Print scraped article description
article_description = soup.find_all('div', class_='rollover_description_inner')[0]
article_description

<div class="rollover_description_inner">
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  
</div>

In [14]:
#Define title and content
article_title = "NASA Ingenuity Mars Helicopter Prepares for First Flight"
paragraph_text = "Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps."

In [15]:
#Pull featured image using splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Captain\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [16]:
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)
browser.links.find_by_partial_text('FULL IMAGE').click()
html = browser.html

In [17]:
soup = BeautifulSoup(html, 'html.parser')
image_link = soup.find('img', class_='fancybox-image')['src']

In [18]:
featured = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{image_link}'
featured

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'

In [19]:
#Pull table from Mars Facts page.
mars_page = pd.read_html('https://space-facts.com/mars/')[0]
mars_page

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [20]:
#Open path to pull cerberus hemisphere images and titles
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
mars_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_url)
site = browser.html
hemispheres = BeautifulSoup(site, 'html.parser')

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Captain\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [21]:
#Narrow down to prep for for loop
results_urls = []

container = hemispheres.find('div', class_= 'result-list')
items = container.find_all('div', class_="item")

In [23]:
#Use for loop to generate information for results_urls dictionary.
for item in items:
    titles = item.find('h3').text
    titles = titles.replace("Enhanced", "")
    links = item.find('a')['href']
    image_link = 'https://astrogeology.usgs.gov/' + links
    browser.visit(image_link)
    new_html = browser.html
    mars_soups = BeautifulSoup(new_html, 'html.parser')
    download = mars_soups.find('div', class_= 'downloads')
    image_urls = download.find('a')['href']
    results_urls.append({'title': titles, 'img_url': image_urls})
    
print(results_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}, {'title': 'Cerberus Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Vi